In [230]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [232]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [233]:
environment = jinja2.Environment()

In [234]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [235]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [236]:
from shutil import copytree, ignore_patterns


copytree('./templates/', './app_output', ignore=ignore_patterns('*.pyc', '__pycache__')) # 'node_modules'


'./app_output'

In [237]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [238]:
typespec

{'reasoning': 'For a gym tracking bot, we need to handle several aspects:\n1. Recording exercises with details like sets, reps, weight\n2. Equipment availability for exercise suggestions\n3. Time constraints for workout planning\n4. Progress tracking over time\n\nExpected message patterns:\n"I did 3 sets of bench press, 8 reps each with 175lbs"\n-> recordExercise({name: "bench press", sets: 3, reps: 8, weight: 175})\n\n"What exercises can I do in 30 minutes with dumbbells and a bench?"\n-> suggestWorkout({duration: "30m", equipment: ["dumbbells", "bench"]})\n\n"Show my bench press progress over the last month"\n-> trackProgress({exercise: "bench press", from: [date], to: [date]})',
 'typespec_definitions': 'model Exercise {\n  name: string;\n  sets: integer;\n  reps: integer;\n  weight: float;\n  notes: string;\n}\n\nmodel Equipment {\n  name: string;\n  type: string;\n}\n\nmodel WorkoutPlan {\n  exercises: Exercise[];\n  totalDuration: duration;\n  difficulty: string;\n}\n\nmodel Prog

In [239]:
print(typespec["reasoning"])

For a gym tracking bot, we need to handle several aspects:
1. Recording exercises with details like sets, reps, weight
2. Equipment availability for exercise suggestions
3. Time constraints for workout planning
4. Progress tracking over time

Expected message patterns:
"I did 3 sets of bench press, 8 reps each with 175lbs"
-> recordExercise({name: "bench press", sets: 3, reps: 8, weight: 175})

"What exercises can I do in 30 minutes with dumbbells and a bench?"
-> suggestWorkout({duration: "30m", equipment: ["dumbbells", "bench"]})

"Show my bench press progress over the last month"
-> trackProgress({exercise: "bench press", from: [date], to: [date]})


In [240]:
print(typespec["typespec_definitions"])

model Exercise {
  name: string;
  sets: integer;
  reps: integer;
  weight: float;
  notes: string;
}

model Equipment {
  name: string;
  type: string;
}

model WorkoutPlan {
  exercises: Exercise[];
  totalDuration: duration;
  difficulty: string;
}

model ProgressRecord {
  exercise: string;
  history: Exercise[];
  improvement: float;
}

interface GymBot {
  @llm_func(2)
  recordExercise(exercise: Exercise): void;

  @llm_func(1)
  suggestWorkout(
    availableEquipment: Equipment[],
    timeLimit: duration,
    fitnessLevel: string
  ): WorkoutPlan;

  @llm_func(3)
  trackProgress(
    exerciseName: string,
    from: utcDateTime,
    to: utcDateTime
  ): ProgressRecord;

  @llm_func(1)
  updateEquipment(equipment: Equipment[]): void;
}


In [241]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [242]:
print(drizzle["drizzle_schema"])

import { 
  integer, 
  pgTable, 
  text, 
  real, 
  timestamp, 
  serial,
  interval,
  primaryKey 
} from "drizzle-orm/pg-core";

export const equipmentTable = pgTable("equipment", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  type: text("type").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const exercisesTable = pgTable("exercises", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight").notNull(),
  notes: text("notes"),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const workoutPlansTable = pgTable("workout_plans", {
  id: serial("id").primaryKey(),
  total_duration: interval("total_duration").notNull(),
  difficulty: text("difficulty").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const workoutPlanExercisesTable = pgTable("workout_plan_exercises", {
 

In [243]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [244]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user, including details like name, sets, reps, weight, and any additional notes about the exercise.',
  'examples': ['I did 3 sets of bench press with 150 lbs',
   'Just finished 4 sets of squats, 12 reps each at 200 pounds',
   'Record my exercise: 3x10 deadlifts at 225 lbs',
   'I want to record my exercise routine for today',
   'Log my workout: did some pull-ups, 3 sets of 8']},
 {'name': 'suggestWorkout',
  'description': "Suggests a workout plan based on available equipment, time constraints, and user's fitness level. Returns a structured workout plan with exercises and duration.",
  'examples': ['What workout can I do with dumbbells in 30 minutes?',
   'Suggest exercises for a beginner with basic home equipment',
   'I have 45 minutes, what workout should I do at the gym?',
   'Create a workout plan for an advanced lifter',
   'What exercises can I do with just a resistance band?']},
 {'name

In [245]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [246]:
print(pre_response.content[0].text)

<instructions>
When processing user input for updateEquipment function:
1. Parse equipment names and types from user messages
2. Each equipment item must have both name and type properties
3. Multiple equipment items should be collected into an array
4. Equipment types should be standardized to common categories: "cardio", "strength", "flexibility", "functional"
5. If type is not specified, infer from common equipment categories
</instructions>

<examples>
    <example>
        <input>Added new treadmill and two dumbbells sets to the gym</input>
        <output>{
  "equipment": [
    {
      "name": "treadmill",
      "type": "cardio"
    },
    {
      "name": "dumbbells",
      "type": "strength"
    }
  ]
}</output>
    </example>

    <example>
        <input>Got new yoga mats and resistance bands</input>
        <output>{
  "equipment": [
    {
      "name": "yoga mat",
      "type": "flexibility"
    },
    {
      "name": "resistance bands",
      "type": "strength"
    }
  ]
}<

In [247]:
pre_processors

{'recordExercise': {'instructions': 'The recordExercise function expects a complete Exercise object with the following properties:\n- name: String describing the exercise (required)\n- sets: Integer number of sets (default to 1 if not specified)\n- reps: Integer number of repetitions (default to 1 if not specified)\n- weight: Float value in kg/lbs (default to 0 if not specified)\n- notes: String for additional comments (default to empty string if not specified)\n\nParse natural language input to extract exercise details. Use pattern matching for common formats like "exercise_name weightxreps" or "exercise_name sets/reps/weight".',
  'examples': [('bench press 80x6',
    '{\n    "name": "bench press",\n    "sets": 1,\n    "reps": 6,\n    "weight": 80.0,\n    "notes": ""\n}'),
   ('did 3 sets of squats at 100kg, 12 reps each',
    '{\n    "name": "squats",\n    "sets": 3,\n    "reps": 12,\n    "weight": 100.0,\n    "notes": ""\n}'),
   ('pushups 3x15 feeling tired today',
    '{\n    "na

In [248]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)


In [255]:
handlers


KeyError: 'instructions'

In [ ]:
from core.interpolator import Interpolator

interpolator = Interpolator('./app_output')

handlers = interpolator.interpolate_all(handlers, pre_processors)


{'recordExercise': {'handler': 'import { db } from "../db";\nimport { exercisesTable, exerciseHistoryTable } from \'../db/schema/application\';\n\nconst handle = async (exercise: Exercise): Promise<void> => {\n    // Record the exercise in both current exercises and history tables\n    await Promise.all([\n        // Insert into main exercises table\n        db.insert(exercisesTable).values({\n            name: exercise.name,\n            sets: exercise.sets,\n            reps: exercise.reps,\n            weight: exercise.weight,\n            notes: exercise.notes\n        }),\n        \n        // Insert into exercise history for tracking progress\n        db.insert(exerciseHistoryTable).values({\n            exercise_name: exercise.name,\n            sets: exercise.sets,\n            reps: exercise.reps,\n            weight: exercise.weight,\n            notes: exercise.notes\n        })\n    ]);\n};',
  'module': 'record_exercise'},
 'suggestWorkout': {'handler': 'import { db } from

In [251]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { exercisesTable, exerciseHistoryTable } from \'../db/schema/application\';\n\nconst handle = async (exercise: Exercise): Promise<void> => {\n    // Record the exercise in both current exercises and history tables\n    await Promise.all([\n        // Insert into main exercises table\n        db.insert(exercisesTable).values({\n            name: exercise.name,\n            sets: exercise.sets,\n            reps: exercise.reps,\n            weight: exercise.weight,\n            notes: exercise.notes\n        }),\n        \n        // Insert into exercise history for tracking progress\n        db.insert(exerciseHistoryTable).values({\n            exercise_name: exercise.name,\n            sets: exercise.sets,\n            reps: exercise.reps,\n            weight: exercise.weight,\n            notes: exercise.notes\n        })\n    ]);\n};',
  'module': 'record_exercise'},
 'suggestWorkout': {'handler': 'import { db } from

In [252]:
with open("./templates/interpolation/router.tpl", "r") as f:
    logic_index_tpl = environment.from_string(f.read())
    
    params = {
        "handlers": handlers
    }
    file_content = logic_index_tpl.render(**params)
    
    with open(os.path.join('./app_output/app_schema/src/logic/router.ts'), 'w') as f:
        f.write(file_content)
            
    print(file_content)

FileNotFoundError: [Errno 2] No such file or directory: './templates/interpolation/router.tpl'

# Project Generation

In [193]:
from shutil import copytree, ignore_patterns

In [194]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [195]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [31]:
from core.stages import router

In [32]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [33]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups. Use this when the user wants to log or record their exercise activity.',
  'examples': ['I want to record my exercise routine for today',
   'I just did 3 sets of bench press',
   'Log my workout: 4 sets of squats with 200lbs',
   'Add my deadlift session to my log']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specific exercise over a given time period. Use this when the user wants to see their progress or history for a particular exercise.',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'What were my deadlift numbers from last week?',
   'Track my progress on pull-ups']},
 {'name': 'suggestWorkout',
  'description': 'Generates a workout plan based on desired duration, target muscle groups, and av

In [47]:
from core.stages import typespec
from core.stages import tsp_compiler

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

typespec_errors = ""
attempts_left = 3

while attempts_left > 0:
    attempts_left -= 1
    
    prompt = tsp_tpl.render(
        application_description=application_description,
        typespec_errors=typespec_errors,
    )
  
    if attempts_left >= 2:
        updated_prompt = prompt + "// PS. Make sure to eject a random error in TypeSpec output for testing purposes"
    else:
        updated_prompt = prompt
  
    tsp_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": updated_prompt}]
    )
    
    typespec = stages.typespec.parse_output(tsp_response.content[0].text)

    # Write TypeSpec definitions to file
    with open('./test/tsp_schema/notebook.tsp', 'w') as f:
        f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        f.write(typespec["typespec_definitions"])

    compiler = tsp_compiler.TypeSpecCompiler('./test/tsp_schema/')
    result = compiler.compile('./notebook.tsp')
    
    print(result)
    
    if result["result"] == tsp_compiler.TypeSpecCompilationStatus.COMPILATION_ERROR:
        typespec_errors = result["errors"]
        print(typespec_errors)
        continue
    else:
        break


In [90]:
typespec


{'reasoning': 'For a finance tracking app, users would likely send messages like:\n- "I spent $50 on groceries yesterday"\n- "Got my salary of $3000 today"\n- "Show me my expenses for last month"\n\nThe LLM would need to extract:\n1. Transaction type (expense/income)\n2. Amount\n3. Category\n4. Date\n5. Optional description/notes\n\nFor reports, users might ask for:\n- Time-based summaries\n- Category-based breakdowns\n- Balance calculations\n\nThe interface should support recording transactions and generating reports while keeping the argument structure simple enough to be extracted from natural language.',
 'typespec_definitions': 'model Transaction {\n  type: string; // "expense" or "income"\n  amount: decimal;\n  category: string;\n  date: utcDateTime;\n  description: string;\n}\n\nmodel TransactionSummary {\n  totalIncome: decimal;\n  totalExpenses: decimal;\n  balance: decimal;\n  transactions: Transaction[];\n}\n\ninterface FinanceTracker {\n  @llm_func(1)\n  recordTransaction(t

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
